In [ ]:
import gym
import numpy as np
import pandas as pd
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick

import os
import sys
sys.path.insert(1, '../agent')
sys.path.insert(1, '../price_pred')

import stock_env
from QLAgent import QNAgent
from helper import *

In [ ]:
name_file_data = 'AAPL.csv'
name_project = 'AXP_batch'
env_name = 'StockEnv-v0'
num_episodes = 10
len_obs = 50
len_window = 100
interval = 1000
load_model = False
epoch_to_load = 10000
name_model_weights = 'trading_weights'
overlap = 20

In [ ]:
path_models, path_imgs, path_stats = check_directories(name_project, len_obs, len_window)
np.random.seed(42)
# Read data you want to use for trading
train_data, test_data = get_data(f'../data/{name_file_data}')

# Create an instant
env = gym.make(env_name, train_data=train_data, eval_data=test_data, len_obs=len_obs, len_window=len_window, init_balance=1000)
print(f'Observation space: {env.observation_space}')
print(f'Action space: {env.action_space}')

# Create an agent
agent = QNAgent(env)

# Load model
if load_model:
    agent.load(path_models + name_model_weights + f'_{epoch_to_load}')
    train_statistics = pd.read_csv(path_stats + 'train.csv')
    test_statistics = pd.read_csv(path_stats + 'test.csv')
    init_ep = epoch_to_load
else:
    init_ep = 0
    train_statistics = pd.DataFrame()
    test_statistics = pd.DataFrame()

for ep in range(init_ep, args.num_episodes):
    get_performance(env, agent, train_data=True, training=True, batch_size=1)
    env.render(ep)

    if (ep % args.interval == 0) and not((args.load_model==True) and (ep == args.epoch_to_load)):
        agent.model.save_weights(path_models + args.name_model_weights + f'_{ep}')

        overlap = args.overlap
        results_train = np.empty(shape=(0, 3))
        results_test = np.empty(shape=(0, 3))

        size_test = ((len(env.eval_data)-env.len_obs-env.len_window) // overlap)+1
        cagr_train, vol_train = get_performance(env, agent, train_data=True, training=False, batch_size=size_test)
        results_train = np.array([np.tile(ep, size_test), cagr_train, vol_train]).transpose()

        cagr_test, vol_test = get_performance(env, agent, train_data=False, training=False, overlap=overlap, batch_size=size_test)
        results_test = np.array([np.tile(ep, size_test), cagr_test, vol_test]).transpose()

        train_statistics = pd.concat([train_statistics, pd.DataFrame(results_train, columns=['epoch', 'cagr','volatility'])])
        train_statistics.to_csv(path_stats+'train.csv', index=False)
        test_statistics = pd.concat([test_statistics, pd.DataFrame(results_test, columns=['epoch', 'cagr','volatility'])])
        test_statistics.to_csv(path_stats+'test.csv', index=False)

        plot_stocks_trading_performance(results_train, path_imgs + f'train_cagr_vol_ep_{ep}',
                                        color='royalblue', acc_title='Train')
        plot_stocks_trading_performance(results_test, path_imgs + f'test_cagr_vol_ep_{ep}',
                                        color='firebrick', acc_title='Test')